# Прогноз максимальной суточной температуры воздуха в Донецке

> Прогноз делается на основании синоптической телеграммы за текущий срок

In [ ]:
#| default_exp core

In [ ]:
#| hide
from nbdev.showdoc import *
import numpy as np
import tensorflow as tf
from datetime import datetime
import time
import ipywidgets as widgets
from ipywidgets import VBox

/home/slavam/my_python/environments/my_env/lib/python3.8/site-packages/fastprogress/fastprogress.py:107: UserWarning: Couldn't import ipywidgets properly, progress bar will use console behavior
  warn("Couldn't import ipywidgets properly, progress bar will use console behavior")
2023-10-04 10:43:34.887702: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-04 10:43:42.612232: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [ ]:
#| hide
def prepare_data(date_term, telegram, model_name):
  ret = []
  # if model_name != 'improve_dnn_model_3':
  #     ret.append(float(telegram[14:15]+'.')) # cloud_height
  #     ret.append(int(telegram[16:17])) # visibility_range
  ret.append(int(telegram[18:19])) # cloud_amount
  sign = ''
  if telegram[25:26] == '1':
    sign = '-'
  ret.append(float(sign+telegram[26:28]+'.'+telegram[28:29]))# temperature
  sign = ''
  if telegram[31:32] == '1':
    sign = '-'
  ret.append(float(sign+telegram[32:34]+'.'+telegram[34:35]))# temperature_dew
  p1 = '1'
  if telegram[37:38] != '0':
    p1 = ''
  ret.append(float(p1+telegram[37:40]+'.'+telegram[40:41]))# pressure
  ret.append(float(telegram[49:50]+'.'))#  pressure_tendency
  ret.append(float(telegram[50:52]+'.'+telegram[52:53]))# pressure_tendency_value
  wv = float(telegram[21:23]+'.')
  wd_rad = int(telegram[19:21])*10*np.pi/180
  ret.append(wv*np.cos(wd_rad)) # Wx
  ret.append(wv*np.sin(wd_rad)) # Wy
  d = datetime.strptime(date_term, "%Y-%m-%d %H:%M:%S")
  s = time.mktime(d.timetuple())
  day = 24*60*60
  year = (365.2425)*day
  ret.append(np.sin(s * (2 * np.pi / day)))
  ret.append(np.cos(s * (2 * np.pi / day)))
  ret.append(np.sin(s * (2 * np.pi / year)))
  ret.append(np.cos(s * (2 * np.pi / year))) #Day sin 	Day cos 	Year sin 	Year cos
  telegram = [ret]
  # print(str(ret[:12]))
  model = tf.keras.models.load_model(model_name)
  predict_temperature = model.predict(telegram).flatten()[0]
  
  return predict_temperature
now = datetime.utcnow()
d_t = now.strftime('%Y-%m-%d')
d_t += ' '+"{:02d}".format(int(now.hour / 3) * 3)+':00:00'
telegram = widgets.Text(
    value='',
    placeholder='Type something',
    description='Последняя телеграмма:',
    disabled=False,
    style={'description_width': 'initial'},
    layout = widgets.Layout(width='800px')
)
def on_click_classify(change):
    max_temperature = prepare_data(d_t, telegram.value, 'improve_dnn_model_3')
    lbl_pred.value = f'Максимальная температура в Донецке за день {d_t[:10]} ожидается {str(round(max_temperature,2))}'
btn_run = widgets.Button(description='Сделать прогноз')
btn_run.on_click(on_click_classify)
lbl_pred = widgets.Label()
VBox([widgets.Label('Прогноз махсимальной температуры в Донецке'), telegram, btn_run, lbl_pred])

In [ ]:
#| export
def foo(): pass

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()